# BERT

### bert_en_UNCASED_L-12_H-768_A-12

In [1]:
import tensorflow_text as text
import tensorflow as tf
import tensorflow_hub as hub
import pandas as pd
import re
import numpy as np

input_file = 'test_data-CLEANED-0.csv' #'train_data-CLEANED-0.csv'
output_file = 'test_data-CLEANED-1.csv' #'train_data-CLEANED-1.csv'

In [2]:
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string)
preprocessor = hub.KerasLayer(
    "bert_en_uncased_preprocess_3")
encoder_inputs = preprocessor(text_input)

encoder = hub.KerasLayer(
    "bert_en_uncased_L-12_H-768_A-12_4",
    trainable=True)
outputs = encoder(encoder_inputs)
pooled_output = outputs["pooled_output"]      # [batch_size, 768].
sequence_output = outputs["sequence_output"]  # [batch_size, seq_length, 768].


In [3]:
# WARNING:tensorflow:Please fix your imports. Module tensorflow.python.training.tracking.data_structures has been moved to tensorflow.python.trackable.data_structures. The old module will be deleted in version 2.11.
# probably no need to worry about this warning

In [4]:
embedding_model = tf.keras.Model(text_input, pooled_output)

In [5]:
# embeddings can be created for both words and sentences
sentences = tf.constant(["Hello World", "Hello"])
embeddings = embedding_model(sentences)
print(embeddings.shape)
print(embeddings[0])

(2, 768)
tf.Tensor(
[-9.05848324e-01 -3.09581727e-01 -6.15243971e-01  7.73076177e-01
  2.85517335e-01 -1.88887998e-01  9.24471915e-01  1.80971816e-01
 -4.99476820e-01 -9.99943674e-01 -2.33899936e-01  8.74012709e-01
  9.77134407e-01  2.53881246e-01  9.37974155e-01 -6.89601123e-01
 -5.18528938e-01 -5.48796833e-01  3.56414378e-01 -7.60025561e-01
  6.04920864e-01  9.99327958e-01  3.36959362e-01  2.51394987e-01
  4.14160699e-01  9.69679117e-01 -7.82691777e-01  9.33998168e-01
  9.63077784e-01  6.59034669e-01 -7.15104818e-01  1.01233251e-01
 -9.85450029e-01 -1.61695302e-01 -6.71174884e-01 -9.85693574e-01
  3.05538833e-01 -7.85906076e-01  1.30508572e-01  1.35026164e-02
 -8.98145497e-01  2.11543441e-01  9.99727130e-01 -1.89671934e-01
  2.36403257e-01 -2.44788393e-01 -9.99999464e-01  2.83703566e-01
 -8.75621021e-01  7.07936943e-01  6.53161883e-01  5.14323711e-01
  1.04176268e-01  4.49569434e-01  4.47404802e-01  2.78437436e-01
 -9.97956023e-02  9.20964852e-02 -2.20018923e-01 -5.42643011e-01
 -6.0

### Embeddings for parts of affiliations

In [6]:
t_data = pd.read_csv(input_file, sep = ";")
t_data.drop(columns = ['Unnamed: 0'], inplace = True)
t_data

,part,labels,affil_index,embeddings_csd-BERT
0,Aventis Pharma,Institution,0,"[-0.6225269436836243, 0.41175007820129395, 0.9..."
1,Romainville,City,0,"[-0.7443768978118896, 0.37689119577407837, 0.9..."
2,France. christophe.dini@aventis.com,Country,0,"[-0.7713785767555237, 0.44986480474472046, 0.9..."
3,Jiangsu Key Laboratory of Drug Design and Opti...,Institution,1,"[-0.615035891532898, 0.31155872344970703, 0.99..."
4,China Pharmaceutical University,Institution,1,"[-0.738187313079834, 0.46809735894203186, 0.99..."
...,...,...,...,...
311,People's Republic of China.,Country,58,"[-0.7836729288101196, 0.5700669884681702, 0.99..."
312,National Center for Natural Products Research,Institution,59,"[-0.6381785273551941, 0.46710556745529175, 0.9..."
313,The University of Mississippi,Institution,59,"[-0.676986813545227, 0.4199213683605194, 0.999..."
314,MS 38677,State,59,"[-0.7675344944000244, 0.5155598521232605, 0.99..."


In [7]:
parts = t_data.iloc[:, 0].tolist()
parts

['Aventis Pharma',
 'Romainville',
 'France. christophe.dini@aventis.com',
 'Jiangsu Key Laboratory of Drug Design and Optimization',
 'China Pharmaceutical University',
 'Nanjing 210009',
 'China',
 'Key Laboratory on Protein Chemistry and Structural Biology',
 'China Pharmaceutical University',
 'Nanjing 210009',
 'China',
 'China Pharmaceutical University',
 'Nanjing 210009',
 'China.',
 'Universidade Federal do Espírito Santo',
 'Rodovia BR 101 Norte',
 'Km 60',
 '29932-900',
 'Brazil.',
 'Jamia Hamdard',
 'New Delhi 110062',
 'India.',
 'Pondicherry University',
 'Puducherry 605 014',
 'India. Electronic address: tchinnasamy@hotmail.com.',
 'Payame Noor University',
 'Khoy 58168-45164',
 'Iran. saghatforoush@gmail.com',
 'Jadavpur University',
 'Kolkata 700 032',
 'India.',
 'Jagiellonian University Medical College',
 'Kraków',
 'Poland. Electronic address: mfmalaws@cyf-kr.edu.pl.',
 'Istituto Superiore di Sanità',
 'Viale Regina Elena 299',
 '00161 Roma',
 'Italy. carlo.mustazza@

In [8]:
#embeddings = embedding_model(tf.constant(parts))
#print(embeddings.shape)

In [9]:
embeddings_parts = []
for part in parts:
    try:
        embeddings_parts.append(embedding_model(tf.constant([part])).numpy().tolist())
    except:
        print(part)
        embeddings_parts.append("X")
t_data["embeddings_uncsd-BERT"] = embeddings_parts
#took 14 mins for 3000 parts of 600 affiliations

In [10]:
t_data

,part,labels,affil_index,embeddings_csd-BERT,embeddings_uncsd-BERT
0,Aventis Pharma,Institution,0,"[-0.6225269436836243, 0.41175007820129395, 0.9...","[[-0.7495080828666687, 0.10450626909732819, 0...."
1,Romainville,City,0,"[-0.7443768978118896, 0.37689119577407837, 0.9...","[[-0.9179261922836304, -0.47461554408073425, -..."
2,France. christophe.dini@aventis.com,Country,0,"[-0.7713785767555237, 0.44986480474472046, 0.9...","[[-0.8849691152572632, -0.5707671642303467, -0..."
3,Jiangsu Key Laboratory of Drug Design and Opti...,Institution,1,"[-0.615035891532898, 0.31155872344970703, 0.99...","[[-0.9479542970657349, -0.6653308272361755, -0..."
4,China Pharmaceutical University,Institution,1,"[-0.738187313079834, 0.46809735894203186, 0.99...","[[-0.9523708820343018, -0.5192119479179382, -0..."
...,...,...,...,...,...
311,People's Republic of China.,Country,58,"[-0.7836729288101196, 0.5700669884681702, 0.99...","[[-0.9732127785682678, -0.5987682938575745, -0..."
312,National Center for Natural Products Research,Institution,59,"[-0.6381785273551941, 0.46710556745529175, 0.9...","[[-0.9274004697799683, -0.2877633571624756, -0..."
313,The University of Mississippi,Institution,59,"[-0.676986813545227, 0.4199213683605194, 0.999...","[[-0.9446776509284973, -0.3148469924926758, -0..."
314,MS 38677,State,59,"[-0.7675344944000244, 0.5155598521232605, 0.99...","[[-0.919567883014679, -0.28708547353744507, -0..."


In [11]:
t_data["embeddings_uncsd-BERT"] = t_data["embeddings_uncsd-BERT"].apply(lambda x: x[0])
t_data

,part,labels,affil_index,embeddings_csd-BERT,embeddings_uncsd-BERT
0,Aventis Pharma,Institution,0,"[-0.6225269436836243, 0.41175007820129395, 0.9...","[-0.7495080828666687, 0.10450626909732819, 0.8..."
1,Romainville,City,0,"[-0.7443768978118896, 0.37689119577407837, 0.9...","[-0.9179261922836304, -0.47461554408073425, -0..."
2,France. christophe.dini@aventis.com,Country,0,"[-0.7713785767555237, 0.44986480474472046, 0.9...","[-0.8849691152572632, -0.5707671642303467, -0...."
3,Jiangsu Key Laboratory of Drug Design and Opti...,Institution,1,"[-0.615035891532898, 0.31155872344970703, 0.99...","[-0.9479542970657349, -0.6653308272361755, -0...."
4,China Pharmaceutical University,Institution,1,"[-0.738187313079834, 0.46809735894203186, 0.99...","[-0.9523708820343018, -0.5192119479179382, -0...."
...,...,...,...,...,...
311,People's Republic of China.,Country,58,"[-0.7836729288101196, 0.5700669884681702, 0.99...","[-0.9732127785682678, -0.5987682938575745, -0...."
312,National Center for Natural Products Research,Institution,59,"[-0.6381785273551941, 0.46710556745529175, 0.9...","[-0.9274004697799683, -0.2877633571624756, -0...."
313,The University of Mississippi,Institution,59,"[-0.676986813545227, 0.4199213683605194, 0.999...","[-0.9446776509284973, -0.3148469924926758, -0...."
314,MS 38677,State,59,"[-0.7675344944000244, 0.5155598521232605, 0.99...","[-0.919567883014679, -0.28708547353744507, -0...."


In [12]:
t_data.loc[0, 'embeddings_uncsd-BERT']

[-0.7495080828666687,
 0.10450626909732819,
 0.8987200260162354,
 0.21800534427165985,
 -0.7901405096054077,
 0.15243056416511536,
 -0.21904200315475464,
 0.04625661298632622,
 0.8810215592384338,
 -0.9961459040641785,
 0.48838382959365845,
 -0.4246334135532379,
 0.9711558818817139,
 -0.7886233925819397,
 0.5324239134788513,
 0.13669686019420624,
 0.3772246539592743,
 -0.15996989607810974,
 0.18815822899341583,
 0.4458329677581787,
 0.018377065658569336,
 -0.8111814260482788,
 0.6634168028831482,
 0.3469413220882416,
 0.053506940603256226,
 -0.8969479203224182,
 -0.03868335485458374,
 0.7097052335739136,
 0.8799365162849426,
 0.6518890857696533,
 0.09897665679454803,
 -0.16476912796497345,
 -0.9858046770095825,
 0.07005993276834488,
 0.8687605857849121,
 -0.9208365082740784,
 -0.2789023518562317,
 -0.5225008726119995,
 0.21088853478431702,
 0.2858724296092987,
 -0.667664110660553,
 0.1158997118473053,
 0.9798393249511719,
 -0.6034476161003113,
 -0.22530552744865417,
 -0.033171158283948

In [13]:
t_data.to_csv(output_file, sep = ';')